In [27]:
import sys, os, importlib, glob
import rasterio as rio
from rasterio.enums import Resampling
# from rasterio.warp import reproject, Resampling
from rasterio.plot import show
import numpy as np
from affine import Affine

import geopandas as gpd
import pandas as pd

from shapely.geometry import box
from shapely.geometry import Polygon, MultiPolygon
from osgeo import gdal
from rasterstats import zonal_stats

from matplotlib import pyplot as plt
import matplotlib.colors as colors
import seaborn as sns

from shapely.wkt import loads

sys.path.append("/home/public/Code/GOST")
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.Urban.UrbanRaster as urban

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Sources
- Worldpop Population Counts (100m)
- WSF Evolution
- Fathom flood maps (fluvial and pluvial)
- Deltares coastal flood maps

In [95]:
countries = {
    'ATG':'Antigua and Barbuda',
    'BHS':'Bahamas',
    'BLZ':'Belize',
    'BRB':'Barbados',
    'DMA':'Dominica',
    'DOM':'Dominican Republic',
    'GRD':'Grenada',
    'GUY':'Guyana',
    'HTI':'Haiti',
    'JAM':'Jamaica',
    'KNA':'Saint Kitts and Nevis',
    'LCA':'Saint Lucia',
    'SUR':'Suriname',
    'MAF':'Saint-Martin', # french
    'SXM':'Sint Maarten', # dutch
    'TCA':'Turks and Caicos Islands', #### DIDN'T RECEIVE WSF DATA
    'TTO':'Trinidad and Tobago',
    'VCT':'Saint Vincent and the Grenadines'    
}

In [96]:
DATA_DIR = '/home/wb514197/data/floods_all'

# Data Preparation

Skip to raster alignment (4. and 5.) 

### 1. Create urban extents

In [4]:
for iso3, country in countries.items():
    out_dir = os.path.join(DATA_DIR,iso3)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    pop_layer_file =   "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/MOSAIC_ppp_prj_2020/ppp_prj_2020_%s.tif" % iso3
    if os.path.exists(pop_layer_file):
        print(f"Creating urban extents for {country}")
        xx = urban.urbanGriddedPop(pop_layer_file)
        densityVal = 2.5 # 300 people per 1,000,000 sq mts (1 sq km)
        urban_extents = xx.calculateVectorUrban(densVal=densityVal, smooth=True, verbose=False, totalPopThresh = 0)
        urban_extents = urban_extents[urban_extents.Pop > 1000]
        urban_extents.to_csv(os.path.join(out_dir, "urban_extents.csv"), index=False)

Creating urban extents for Antigua and Barbuda
Creating urban extents for Bahamas, The
Creating urban extents for Belize
Creating urban extents for Barbados
Creating urban extents for Dominica
Creating urban extents for Dominican Republic
Creating urban extents for Grenada
Creating urban extents for Guyana
Creating urban extents for Haiti
Creating urban extents for Jamaica
Creating urban extents for St. Kitts and Nevis
Creating urban extents for St. Lucia
Creating urban extents for Suriname
Creating urban extents for Turks and Caicos Islands
Creating urban extents for Trinidad and Tobago
Creating urban extents for St. Vincent and the Grenadines


### 2. Create extents for WSF request

In [35]:
all_bounds = pd.DataFrame()

In [36]:
for iso3, country in countries.items():
    out_dir = os.path.join(DATA_DIR,iso3)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    pop_layer_file = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/MOSAIC_ppp_prj_2020/ppp_prj_2020_%s.tif" % iso3
    if os.path.exists(pop_layer_file):
        pop = rio.open(pop_layer_file)
        pop_bounds = box(*pop.bounds)
        all_bounds.loc[country, "iso3"] = iso3
        all_bounds.loc[country, "left"] = pop.bounds.left
        all_bounds.loc[country, "bottom"] = pop.bounds.bottom
        all_bounds.loc[country, "right"] = pop.bounds.right
        all_bounds.loc[country, "top"] = pop.bounds.top

In [40]:
all_bounds.to_csv("Extents_for_WSF.csv")

### 3. Prepare Deltares flood files

In [91]:
coastal_dir = '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020'
periods_coast = [5, 50, 100]

In [44]:
for iso3, country in countries.items():
    out_dir = os.path.join(DATA_DIR, iso3)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    out_coastal = os.path.join(out_dir, 'coastal')
    if not os.path.exists(out_coastal):
        os.mkdir(out_coastal)
    country_dirs = [x for x in os.listdir(coastal_dir) if iso3 in x]
    if country=="Belize":
        for dir_name in country_dirs:
            if "localDEM" in dir_name:
                country_dirs.remove(dir_name)
#     if len(country_dirs)==1:
    print(country)
    out_files = []
    for rp in periods_coast:
        files = []
        for folder in country_dirs:
            data_path = os.path.join(coastal_dir, folder, 'Flood_maps_current_climate')
            file = glob.glob(data_path+f"/*RP_{rp}.tif")[0]
            files.append(file)
        input_files = "'"+"' '".join(files)+"'"
        out_file = os.path.join(out_dir, 'coastal', f"{iso3}_{rp}.tif")
        out_files+=[out_file]
        merge_command = f"gdal_merge.py -o {out_file} -of GTiff -a_nodata -99999 {input_files}"
        print(merge_command)
        print("\n")

Antigua and Barbuda
gdal_merge.py -o /home/wb514197/data/floods_all/ATG/coastal/ATG_5.tif -of GTiff -a_nodata -99999 '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020/Antigua and BarbudaATG001_res90_MERIT/Flood_maps_current_climate/AntiguaandBarbudaATG001RCP452010_epsg_32620_RP_5.tif' '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020/Antigua and BarbudaATG002_res90_MERIT/Flood_maps_current_climate/AntiguaandBarbudaATG002RCP452010_epsg_32620_RP_5.tif' '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020/Antigua and BarbudaATG003_res90_MERIT/Flood_maps_current_climate/AntiguaandBarbudaATG003RCP452010_epsg_32620_RP_5.tif' '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020/Antigua and BarbudaATG004_res90_MERIT/Flood_maps_current_climate/AntiguaandBarbudaATG004RCP452010_epsg_32620_RP_5.tif'


gdal_merge.py -o /home/wb514197/data/floods_all/A

gdal_merge.py -o /home/wb514197/data/floods_all/HTI/coastal/HTI_5.tif -of GTiff -a_nodata -99999 '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020/HaitiHTI001_res90_MERIT/Flood_maps_current_climate/HaitiHTI001RCP452010_epsg_32618_RP_5.tif' '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020/HaitiHTI002_res90_MERIT/Flood_maps_current_climate/HaitiHTI002RCP452010_epsg_32618_RP_5.tif' '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020/HaitiHTI003_res90_MERIT/Flood_maps_current_climate/HaitiHTI003RCP452010_epsg_32618_RP_5.tif' '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020/HaitiHTI004_res90_MERIT/Flood_maps_current_climate/HaitiHTI004RCP452010_epsg_32618_RP_5.tif' '/home/public/Data/GLOBAL/FLOOD_SSBN/01_Current_climate_floodmaps_all_countries_03042020/HaitiHTI005_res90_MERIT/Flood_maps_current_climate/HaitiHTI005RCP452010_epsg_32618_RP_

In [97]:
for iso3, country in countries.items():
    out_dir = os.path.join(DATA_DIR, iso3)
    out_coastal = os.path.join(out_dir, 'coastal')
    out_coastal_wgs = os.path.join(out_coastal, 'wgs')
    if not os.path.exists(out_coastal_wgs):
        os.mkdir(out_coastal_wgs)
    print(country)
    for rp in periods_coast:
        out_file = os.path.join(out_dir, 'coastal', f"{iso3}_{rp}.tif")
        file_name = out_file.split("/")[-1]
        out_proj = os.path.join(out_coastal_wgs, file_name)
        print(f"reprojecting {out_file}")
        gdal.Warp(out_proj, out_file, dstSRS='EPSG:4326')
        print("\n")

Antigua and Barbuda
reprojecting /home/wb514197/data/floods_all/ATG/coastal/ATG_5.tif


reprojecting /home/wb514197/data/floods_all/ATG/coastal/ATG_50.tif


reprojecting /home/wb514197/data/floods_all/ATG/coastal/ATG_100.tif


Bahamas
reprojecting /home/wb514197/data/floods_all/BHS/coastal/BHS_5.tif


reprojecting /home/wb514197/data/floods_all/BHS/coastal/BHS_50.tif


reprojecting /home/wb514197/data/floods_all/BHS/coastal/BHS_100.tif


Belize
reprojecting /home/wb514197/data/floods_all/BLZ/coastal/BLZ_5.tif


reprojecting /home/wb514197/data/floods_all/BLZ/coastal/BLZ_50.tif


reprojecting /home/wb514197/data/floods_all/BLZ/coastal/BLZ_100.tif


Barbados
reprojecting /home/wb514197/data/floods_all/BRB/coastal/BRB_5.tif


reprojecting /home/wb514197/data/floods_all/BRB/coastal/BRB_50.tif


reprojecting /home/wb514197/data/floods_all/BRB/coastal/BRB_100.tif


Dominica
reprojecting /home/wb514197/data/floods_all/DMA/coastal/DMA_5.tif


reprojecting /home/wb514197/data/floods_all/DMA/coa

TO DO: Download World Pop files (doesn't work)

In [102]:
# from urllib.request import urlretrieve, urlopen
# from urllib.error import HTTPError, URLError
# import sys, os

In [104]:
# iso3 = 'VCT'
# out_dir = os.path.join(DATA_DIR, iso3)
# out_wp = os.path.join(out_dir, "worldpop")
# if not os.path.exists(out_wp):
#     os.mkdir(out_wp)
#     for year in range(2000, 2025, 5):
#         url = f"ftp://ftp.worldpop.org.uk/GIS/Population/Global_2000_2020/{year}/VCT/vct_ppp_{year}_UNadj.tif"
#         target = os.path.join(out_wp, os.path.basename(url))
#         urlretrieve(url, target)

URLError: <urlopen error ftp error: TimeoutError(110, 'Connection timed out')>

In [ ]:
# for year in range(2000, 2025, 5):
#     url = f"ftp://ftp.worldpop.org.uk/GIS/Population/Global_2000_2020/{year}/VCT/vct_ppp_{year}_UNadj.tif"
#     target = os.path.join(pop_folder, os.path.basename(url))
#     urlretrieve(url, target)

### 4. Check WSF / Fathom paths

The names of folders change slightly by country. This also tracks data avialability

In [98]:
fathom_meta = {
    'ATG':'Antigua & Barbuda',
    'BHS':'Bahamas',
    'BLZ':'Belize',
    'BRB':'Barbados',
    'DMA':'Dominica',
    'DOM':'Dominican Republic',
    'GRD':'Grenada',
    'GUY':'Guyana',
    'HTI':'Haiti',
    'JAM':'Jamaica',
    'KNA': None, #'Saint Kitts and Nevis',
    'LCA':'St Lucia',
    'SXM': None, #'Sint Maarten', # dutch
    'SUR':'Suriname',
    'MAF': None, #'Saint-Martin', # french
    'TCA':'Turks and Caicos Islands',
    'TTO':'Trinidad and Tobago',
    'VCT':'St Vincent and the Grenadines'
}

In [99]:
wsf_meta = {
    'ATG':'AntiguaBarbuda',
    'BHS':'TheBahamas',
    'BLZ':'Belize',
    'BRB':'Barbados',
    'DMA':'Dominica',
    'DOM':'DominicanRepublic',
    'GRD':'Grenada',
    'GUY':'Guyana',
    'HTI':'Haiti',
    'JAM':'Jamaica',
    'KNA':'StKittsNevis', #'Saint Kitts and Nevis',
    'LCA':'SaintLucia',
    'SXM':'SintMaarten', #'Sint Maarten', # dutch
    'SUR':'Suriname', 
    'MAF':'StMartin', #'Saint-Martin', # french
    'TCA': None,
    'TTO': 'TrinidadTobago',
    'VCT':'SaintVincentAndTheGrenadines'
}

In [100]:
fathom_dir = '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019'
wsf_dir = '/home/public/Data/GLOBAL/WSF'

### 5. Raster alignment + stack together

In [101]:
raster_stack_meta = {}

In [102]:
for iso3, country in countries.items():
    out_dir = os.path.join(DATA_DIR,iso3)
    if wsf_meta[iso3] is not None:
        print(f"Stacking rasters for {country}")
        data_bands = {}
        band_count = 0

        periods = ['1in5', '1in50','1in100', '1in1000']
        flood_types = ['fluvial', 'pluvial']
        
        wsf_file = os.path.join(wsf_dir, f"{wsf_meta[iso3]}_WSF_evolution.tif")
        
        flood_files = []
        if fathom_meta[iso3] is not None:
            flood_path = os.path.join(fathom_dir, f"{fathom_meta[iso3]}")
            fathom_data = os.listdir(flood_path)
            for period in periods:
                for flood_type in flood_types:
                    if f"{flood_type}_undefended" in fathom_data:
                        flood_raster = os.path.join(flood_path, f"{flood_type}_undefended", f"{flood_type[0].upper()}U_{period}.tif")
                        flood_files.append(flood_raster)
                        band_count+=1
                        data_bands[('flood data',period,flood_type)] = band_count
            #             data_bands['flood data'][period][flood_type] = band_count
                    elif flood_type in fathom_data:
                        flood_raster = os.path.join(flood_path, flood_type, f"{flood_type[0].upper()}_{period}.tif")
                        flood_files.append(flood_raster)
                        band_count+=1
                        data_bands[('flood data',period,flood_type)] = band_count
            #             data_bands['flood data'][period][flood_type] = band_count
                    else:
                        print(f"No {period} {flood_type} data for {country}")
        else:
            print(f"no fathom flood data for {country}")

        out_coastal_wgs = os.path.join(out_dir, 'coastal', 'wgs')
        coastal_files = [os.path.join(out_coastal_wgs, x) for x in os.listdir(out_coastal_wgs)]
#         coastal_files.sort()
        for period in periods[0:3]:
            band_count+=1
        #     data_bands['flood data'][period]['coastal'] = band_count
            data_bands[('flood data',period,'coastal')] = band_count

        out_wp = os.path.join(out_dir, "worldpop")
        pop_annual_files = [os.path.join(out_wp, x) for x in os.listdir(out_wp)]
        pop_annual_files.sort()
        for pop_year in range(2000,2025,5):
            band_count+=1
        #     data_bands['population'][str(pop_year)] = band_count
            data_bands[('population',str(pop_year))] = band_count

        # wsf
        band_count+=1
        # data_bands['wsf'] = band_count
        data_bands[('wsf')] = band_count
        raster_stack_meta[iso3] = data_bands
        input_files = flood_files+coastal_files+pop_annual_files+[wsf_file]
        print(input_files)
        print("\n")

        # Generate stacked Tif from input bands, only needs to be run once
        out_vrt = os.path.join(out_dir, "combo.vrt")
        out_tif = out_vrt.replace(".vrt", ".tif")
        outds = gdal.BuildVRT(out_vrt, input_files, separate=True, resolution='highest')
        outds = gdal.Translate(out_tif, outds)

Stacking rasters for Antigua and Barbuda
['/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Antigua & Barbuda/fluvial_undefended/FU_1in5.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Antigua & Barbuda/pluvial/P_1in5.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Antigua & Barbuda/fluvial_undefended/FU_1in50.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Antigua & Barbuda/pluvial/P_1in50.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Antigua & Barbuda/fluvial_undefended/FU_1in100.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Antigua & Barbuda/pluvial/P_1in100.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Antigua & Barbuda/fluvial_undefended/FU_1in1000.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Antigua & Barbuda/pluvial/P_1in1000.tif', '/home/wb514197/data/floods_all/ATG/coastal/wgs/ATG_5.tif', '/home/wb514197/data/floods_all/ATG/coastal/wgs/ATG_50.tif', '/home/wb514197/data/floods_all/ATG/coastal/wgs/ATG_100.tif', '/home/wb514197/data/floods_all/ATG/worldpo

Stacking rasters for Guyana
['/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Guyana/fluvial_undefended/FU_1in5.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Guyana/pluvial/P_1in5.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Guyana/fluvial_undefended/FU_1in50.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Guyana/pluvial/P_1in50.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Guyana/fluvial_undefended/FU_1in100.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Guyana/pluvial/P_1in100.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Guyana/fluvial_undefended/FU_1in1000.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/Guyana/pluvial/P_1in1000.tif', '/home/wb514197/data/floods_all/GUY/coastal/wgs/GUY_5.tif', '/home/wb514197/data/floods_all/GUY/coastal/wgs/GUY_50.tif', '/home/wb514197/data/floods_all/GUY/coastal/wgs/GUY_100.tif', '/home/wb514197/data/floods_all/GUY/worldpop/guy_ppp_2000_UNadj.tif', '/home/wb514197/data/floods_all/GUY/worldpop/guy_ppp_2005_UNadj.tif', '/ho

Stacking rasters for Saint Vincent and the Grenadines
No 1in5 fluvial data for Saint Vincent and the Grenadines
No 1in50 fluvial data for Saint Vincent and the Grenadines
No 1in100 fluvial data for Saint Vincent and the Grenadines
No 1in1000 fluvial data for Saint Vincent and the Grenadines
['/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/St Vincent and the Grenadines/pluvial/P_1in5.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/St Vincent and the Grenadines/pluvial/P_1in50.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/St Vincent and the Grenadines/pluvial/P_1in100.tif', '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019/St Vincent and the Grenadines/pluvial/P_1in1000.tif', '/home/wb514197/data/floods_all/VCT/coastal/wgs/VCT_5.tif', '/home/wb514197/data/floods_all/VCT/coastal/wgs/VCT_50.tif', '/home/wb514197/data/floods_all/VCT/coastal/wgs/VCT_100.tif', '/home/wb514197/data/floods_all/VCT/worldpop/vct_ppp_2000_UNadj.tif', '/home/wb514197/data/floods_all/VCT/worldpop/vct_ppp_2005_UNadj.

# Data Analysis

### A)  Urban Exposure
1. Loop through each return period
2. Define flooded areas with several min depth (see flood_bins)
3. Summarize flooded areas that intersect the WSF dataset (built-up by 5-year period)

In [202]:
selected_countries = {
    'ATG':'Antigua and Barbuda',
    'BHS':'Bahamas',
    'BLZ':'Belize',
    'BRB':'Barbados',
    'DMA':'Dominica',
    'DOM':'Dominican Republic',
    'GRD':'Grenada',
    'GUY':'Guyana',
    'HTI':'Haiti',
    'JAM':'Jamaica',
    'KNA':'Saint Kitts and Nevis',
    'LCA':'Saint Lucia',
    'SUR':'Suriname',
    'MAF':'Saint-Martin', # french
    'SXM':'Sint Maarten', # dutch
#     'TCA':'Turks and Caicos Islands', #### DIDN'T RECEIVE WSF DATA
    'TTO':'Trinidad and Tobago',
    'VCT':'Saint Vincent and the Grenadines'    
}

In [203]:
def load_csv(path, geometry='geometry', crs = 'epsg:4326'):
    df = pd.read_csv(path)
    df[geometry] = df[geometry].apply(lambda x: loads(x))
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    return gdf

In [204]:
wsf_dict = {
    0:'non-flooded',
    1:"1985-90",
    2:"1990-95",
    3:"1995-00",
    4:"2000-05",
    5:"2005-10",
    6:"2010-15"
}
flood_bins = {
    0: '>0',
    0.15: '>15cm',
    0.5: '>50cm',
    1: '>1m'
}

In [205]:
results_all = {}

In [206]:
for iso3, country in selected_countries.items():
    print(f"Processing {country}")
    out_dir = os.path.join(DATA_DIR,iso3)
    out_tif = os.path.join(out_dir, "combo.tif")
    cur_tif = rio.open(out_tif)

    # Load metadata and urban extents
    data_bands = raster_stack_meta[iso3]
    dst_transform = cur_tif.transform
    urban_extents = load_csv(os.path.join(out_dir, "urban_extents.csv"))
    urban_extents = urban_extents[urban_extents.Pop > 1000]

    # to store country results
    results_wsf = {}
    zs_tot_country = []
    for period in periods:
        results_wsf[period] = []

    # iterate by urban area, much more efficient since they are islands
    for idx, row in urban_extents.iterrows():

        geometry = row['geometry']
        # get pixel coordinates of the geometry's bounding box
        ul = cur_tif.index(*geometry.bounds[0:2])
        lr = cur_tif.index(*geometry.bounds[2:4])
        window = ((float(lr[0]), float(ul[0]+1)), (float(ul[1]), float(lr[1]+1)))
        t = dst_transform
        shifted_affine = Affine(t.a, t.b, t.c+ul[1]*t.a, t.d, t.e, t.f+lr[0]*t.e)
        
        # reorganize data
        data = {
                'flood data':{
                    '1in5': {
                        'fluvial': cur_tif.read(data_bands[('flood data','1in5','fluvial')], window=window) if ('flood data','1in5','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in5','pluvial')], window=window) if ('flood data','1in5','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in5','coastal')], window=window) if ('flood data','1in5','coastal') in data_bands else None
                    },
                    '1in50':{
                        'fluvial': cur_tif.read(data_bands[('flood data','1in50','fluvial')], window=window) if ('flood data','1in50','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in50','pluvial')], window=window) if ('flood data','1in50','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in50','coastal')], window=window) if ('flood data','1in50','coastal') in data_bands else None
                    },
                    '1in100': {
                        'fluvial': cur_tif.read(data_bands[('flood data','1in100','fluvial')], window=window) if ('flood data','1in100','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in100','pluvial')], window=window) if ('flood data','1in100','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in100','coastal')], window=window) if ('flood data','1in100','coastal') in data_bands else None
                    },
                    '1in1000': {
                        'fluvial': cur_tif.read(data_bands[('flood data','1in1000','fluvial')], window=window) if ('flood data','1in1000','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in1000','pluvial')], window=window) if ('flood data','1in1000','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in1000','coastal')], window=window) if ('flood data','1in1000','coastal') in data_bands else None
                    }
                },
                'population': {
                    '2000': cur_tif.read(data_bands[('population','2000')], window=window),
                    '2005': cur_tif.read(data_bands[('population','2005')], window=window),
                    '2010': cur_tif.read(data_bands[('population','2010')], window=window),
                    '2015': cur_tif.read(data_bands[('population','2015')], window=window),
                    '2020': cur_tif.read(data_bands[('population','2020')], window=window)
                },
                'wsf': cur_tif.read(data_bands['wsf'], window=window)
            }

        print(f"\tLoaded data for {country}, ID: {row.ID}")

        for period in periods:
            
            if (pluvial is None and fluvial is None) and period =='1in1000':
                continue
            
            print(f"\t\tprocessing {period}")
            try:
                # Prepare flood data
                fluvial = data['flood data'][period]['fluvial']
                pluvial = data['flood data'][period]['pluvial']
                coastal = data['flood data'][period]['coastal']

                if pluvial is None and fluvial is None:
                    coastal[(coastal == -99999.0)] = -9999
                    max_depth = coastal.copy()
                else:
                    if pluvial is not None:
                        pluvial[(pluvial == -9999.) | (pluvial == 999)] = -9999
                    if fluvial is not None:
                        fluvial[(fluvial == -9999.) | (fluvial == 999)] = -9999
                        max_fathom = np.maximum(fluvial, pluvial)
                    else:
                        max_fathom = pluvial.copy()

                    if coastal is not None:
                        coastal[(coastal == -99999.0)] = -9999
                        max_depth = np.maximum(max_fathom, coastal)
                    else:
                        max_depth = max_fathom.copy()
                max_depth_ma = np.ma.array(max_depth, mask=(max_depth==-9999), fill_value=-9999)

                # Prepare wsf data
                wsf = data['wsf']
                wsf_bins = [1985, 1990, 1995, 2000, 2005, 2010, 2016]
                # first bin includes all zeros
                wsf_cat = np.digitize(wsf, wsf_bins)
                wsf_cat_ma = np.ma.array(wsf_cat, mask=(wsf_cat==0), fill_value=-9999)

                if (max_depth_ma.shape != wsf_cat.shape):
                    print("error: datasets do not have the same shape")

                results_dict = {}
                # for each min flood, get sum of cells intersecting flood categories
                for key, label in flood_bins.items():
                    flood_bool = (max_depth_ma>key)*1
                    flood_wsf_intersect = wsf_cat_ma*flood_bool
                    zs = pd.DataFrame(zonal_stats(row.geometry, flood_wsf_intersect.filled(), affine=shifted_affine, categorical=True, category_map=wsf_dict, nodata=-9999))
                    zs.index = [row.ID]
                    results_dict[label] = zs

                results_df = pd.concat(results_dict, sort=False)
                results_df.fillna(0, inplace = True)

                results_wsf[period].append(results_df)
            except:
                print(f"failed at period {period}")

        # prepare sum of built up to get shares
        zs_tot = pd.DataFrame(zonal_stats(row.geometry, wsf_cat_ma.filled(), affine=shifted_affine, categorical=True, category_map=wsf_dict, nodata=-9999))
        zs_tot.fillna(0, inplace = True)
        zs_tot.index = [row.ID]
        zs_tot_country.append(zs_tot)

    # AGGREGATE TO COUNTRY-LEVEL
    
    # convert list of df into one df
    for key, item in results_wsf.items():
        if (pluvial is None and fluvial is None) and key =='1in1000':
            results_wsf[key] = None
            continue
        results_wsf[key] = pd.concat(results_wsf[key]).groupby(level=0).sum()
    results_wsf_df = pd.concat(results_wsf, sort=False)
    for wsf_year in wsf_dict.values():
        if wsf_year not in results_wsf_df.columns:
            results_wsf_df[wsf_year] = 0
    results_wsf_df = results_wsf_df[wsf_dict.values()]
    results_wsf_df.index.names = ['return period', 'min depth']
    results_wsf_df.fillna(0, inplace = True)
    results_wsf_df = results_wsf_df.iloc[:, 1:].cumsum(axis=1)

    results_wsf_df['country'] = country
    results_wsf_df = results_wsf_df.reset_index()
    results_wsf_df = results_wsf_df.set_index(['country', 'return period', 'min depth'])

    # aggregate sum of built-up
    results_tot = pd.DataFrame(pd.concat(zs_tot_country).sum(), columns=[country]).transpose()
    for wsf_year in wsf_dict.values():
        if wsf_year not in results_tot.columns:
            results_tot[wsf_year] = 0
    results_tot = results_tot[list(wsf_dict.values())[1:]]
    results_tot = results_tot.cumsum(axis=1)
    results_tot.index.name = 'country'

    # get shares
    results_pct = results_wsf_df.div(results_tot)
    results_pct = results_pct.reset_index(drop=False).set_index('country')
    results_all[iso3] = results_pct

Processing Antigua and Barbuda
	Loaded data for Antigua and Barbuda, ID: 5
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Antigua and Barbuda, ID: 6
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Antigua and Barbuda, ID: 8
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
Processing Bahamas
	Loaded data for Bahamas, ID: 26
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Bahamas, ID: 30
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Bahamas, ID: 35
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
Processing Belize
	Loaded data for Belize, ID: 0
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Belize, ID: 1
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data fo

	Loaded data for Dominican Republic, ID: 146
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 149
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 150
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 151
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 156
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 157
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 162
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 165
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for

		processing 1in1000
	Loaded data for Dominican Republic, ID: 363
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 367
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 368
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 369
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 374
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 376
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 378
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 382
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in

	Loaded data for Dominican Republic, ID: 542
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 548
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 549
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 550
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 551
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 553
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 554
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 558
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for

		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 423
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 434
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 444
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 445
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 447
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 454
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 457
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 458
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 475
		processing 1in5
		processing 1in50
		processi

	Loaded data for Haiti, ID: 1485
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
Processing Jamaica
	Loaded data for Jamaica, ID: 1
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Jamaica, ID: 2
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Jamaica, ID: 3
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Jamaica, ID: 5
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Jamaica, ID: 7
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Jamaica, ID: 8
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Jamaica, ID: 11
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Jamaica, ID: 12
		processing 1in5
		processing 1in50
		processing 1in100
		process

		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 7
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 9
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 10
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 13
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 17
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
Processing Suriname
	Loaded data for Suriname, ID: 1
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Suriname, ID: 6
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Suriname, ID: 8
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data f

In [207]:
results_all_df = pd.concat(results_all, sort=False)

In [208]:
results_all_df.tail()

return period min depth   1985-90  \
    country                                                              
VCT Saint Vincent and the Grenadines        1in100     >50cm  0.063755   
    Saint Vincent and the Grenadines       1in1000        >0  0.310366   
    Saint Vincent and the Grenadines       1in1000     >15cm  0.216039   
    Saint Vincent and the Grenadines       1in1000       >1m  0.041787   
    Saint Vincent and the Grenadines       1in1000     >50cm  0.101356   

                                       1990-95   1995-00   2000-05   2005-10  \
    country                                                                    
VCT Saint Vincent and the Grenadines  0.063804  0.063771  0.063645  0.063843   
    Saint Vincent and the Grenadines  0.310351  0.310314  0.308264  0.307486   
    Saint Vincent and the Grenadines  0.216045  0.215941  0.214395  0.213783   
    Saint Vincent and the Grenadines  0.041782  0.041825  0.041451  0.041587   
    Saint Vincent and the Grenadines  0.101443  0.101421  0.101007  0.101627   

                                       2010-15  
    country                                     
VCT Saint Vincent and the Grenadines  0.064145  
    Saint Vincent and the Grenadines  0.307500  
    Saint Vincent and the Grenadines  0.212810  
    Saint Vincent and the Grenadines  0.042308  
    Saint Vincent and the Grenadines  0.102099

In [209]:
results_all_df.to_csv("urban_exposed_to_floods_share_of_built_AUG7.csv")

### B)  Exposed population
1. Loop through each return period
2. Define flooded areas with several min depth (see flood_bins)
3. Summarize urban population in flooded areas at 5-year intervals

In [136]:
results_all_pop = {}

In [137]:
for iso3, country in selected_countries.items():
    print(f"Processing {country}")
    out_dir = os.path.join(DATA_DIR,iso3)
    out_tif = os.path.join(out_dir, "combo.tif")
    cur_tif = rio.open(out_tif)

    # Load metadata and urban extents
    data_bands = raster_stack_meta[iso3]
    dst_transform = cur_tif.transform
    urban_extents = load_csv(os.path.join(out_dir, "urban_extents.csv"))
    urban_extents = urban_extents[urban_extents.Pop > 1000]

    # to store country results
    results_pop = {}
    for period in periods:
        results_pop[period] = []
    zs_sum_country = []
    
    # iterate by urban area, much more efficient since they are islands
    for idx, row in urban_extents.iterrows():

        geometry = row['geometry']
        # get pixel coordinates of the geometry's bounding box
        ul = cur_tif.index(*geometry.bounds[0:2])
        lr = cur_tif.index(*geometry.bounds[2:4])
        window = ((float(lr[0]), float(ul[0]+1)), (float(ul[1]), float(lr[1]+1)))
        t = dst_transform
        shifted_affine = Affine(t.a, t.b, t.c+ul[1]*t.a, t.d, t.e, t.f+lr[0]*t.e)
        
        # reorganize data
        data = {
                'flood data':{
                    '1in5': {
                        'fluvial': cur_tif.read(data_bands[('flood data','1in5','fluvial')], window=window) if ('flood data','1in5','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in5','pluvial')], window=window) if ('flood data','1in5','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in5','coastal')], window=window) if ('flood data','1in5','coastal') in data_bands else None
                    },
                    '1in50':{
                        'fluvial': cur_tif.read(data_bands[('flood data','1in50','fluvial')], window=window) if ('flood data','1in50','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in50','pluvial')], window=window) if ('flood data','1in50','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in50','coastal')], window=window) if ('flood data','1in50','coastal') in data_bands else None
                    },
                    '1in100': {
                        'fluvial': cur_tif.read(data_bands[('flood data','1in100','fluvial')], window=window) if ('flood data','1in100','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in100','pluvial')], window=window) if ('flood data','1in100','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in100','coastal')], window=window) if ('flood data','1in100','coastal') in data_bands else None
                    },
                    '1in1000': {
                        'fluvial': cur_tif.read(data_bands[('flood data','1in1000','fluvial')], window=window) if ('flood data','1in1000','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in1000','pluvial')], window=window) if ('flood data','1in1000','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in1000','coastal')], window=window) if ('flood data','1in1000','coastal') in data_bands else None
                    }
                },
                'population': {
                    '2000': cur_tif.read(data_bands[('population','2000')], window=window),
                    '2005': cur_tif.read(data_bands[('population','2005')], window=window),
                    '2010': cur_tif.read(data_bands[('population','2010')], window=window),
                    '2015': cur_tif.read(data_bands[('population','2015')], window=window),
                    '2020': cur_tif.read(data_bands[('population','2020')], window=window)
                },
                'wsf': cur_tif.read(data_bands['wsf'], window=window)
            }

        print(f"\tLoaded data for {country}, ID: {row.ID}")

        for period in periods:

            if (pluvial is None and fluvial is None) and period =='1in1000': # if no fathom data can't run this RP
                continue
            
            print(f"\t\tprocessing {period}")

            # Prepare flood data
            fluvial = data['flood data'][period]['fluvial']
            pluvial = data['flood data'][period]['pluvial']
            coastal = data['flood data'][period]['coastal']

            if pluvial is None and fluvial is None:
                coastal[(coastal == -99999.0)] = -9999
                max_depth = coastal.copy()
            else:
                if pluvial is not None:
                    pluvial[(pluvial == -9999.) | (pluvial == 999)] = -9999
                if fluvial is not None:
                    fluvial[(fluvial == -9999.) | (fluvial == 999)] = -9999
                    max_fathom = np.maximum(fluvial, pluvial)
                else:
                    max_fathom = pluvial.copy()

                if coastal is not None:
                    coastal[(coastal == -99999.0)] = -9999
                    max_depth = np.maximum(max_fathom, coastal)
                else:
                    max_depth = max_fathom.copy()
            max_depth_ma = np.ma.array(max_depth, mask=(max_depth==-9999), fill_value=-9999)
            
            results_dict = {}
            # for each min flood, get sum of population intersecting flood categories
            for key, label in flood_bins.items():
                flood_bool = (max_depth_ma>key)*1
                zs_pop = None
                for pop_year in data['population'].keys():
                    pop = data['population'][pop_year]
                    pop[pop==-99999.] = -9999
                    pop_flood_intersect = pop*flood_bool
                    zs = pd.DataFrame(zonal_stats(row.geometry, pop_flood_intersect.filled(), affine=shifted_affine, stats='sum', nodata=-9999)).rename(columns = {'sum':pop_year})
                    zs.index = [row.ID]
                    if zs_pop is None:
                        zs_pop = zs
                    else:
                        zs_pop = zs_pop.join(zs)
                results_dict[label] = zs_pop

            results_df = pd.concat(results_dict, sort=False)
            results_df.fillna(0, inplace = True)

            results_pop[period].append(results_df)

        # get total urban population per year
        sum_pop = {}
        zs_sum = None
        for pop_year in data['population'].keys():
            pop = data['population'][pop_year]
            pop[pop==-99999.] = -9999
            zs = pd.DataFrame(zonal_stats(row.geometry, pop, affine=shifted_affine, stats='sum', nodata=-9999)).rename(columns = {'sum':pop_year})
            if zs_sum is None:
                zs_sum = zs
            else:
                zs_sum = zs_sum.join(zs)
        zs_sum.index = [row.ID]
        zs_sum_country.append(zs_sum)

    # AGGREGATE TO COUNTRY-LEVEL
    
    # convert list of df into one df
    for key, item in results_pop.items():
        if (pluvial is None and fluvial is None) and key =='1in1000':
            results_pop[key] = None
            continue
        results_pop[key] = pd.concat(results_pop[key]).groupby(level=0).sum()
    results_pop_df = pd.concat(results_pop, sort=False)
    results_pop_df.index.names = ['return period', 'min depth']
    results_pop_df['country'] = country
    results_pop_df = results_pop_df.reset_index(drop=False).set_index('country')
            
    # sum of urban pop
    zs_sum_country = pd.DataFrame(pd.concat(zs_sum_country).sum(), columns=[country]).transpose()

    # get shares
    results_pct = pd.concat([results_pop_df[['min depth','return period']], results_pop_df[data['population'].keys()]/zs_sum_country], axis=1)
    results_all_pop[iso3] = results_pct

Processing Antigua and Barbuda
	Loaded data for Antigua and Barbuda, ID: 5
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Antigua and Barbuda, ID: 6
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Antigua and Barbuda, ID: 8
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
Processing Bahamas
	Loaded data for Bahamas, ID: 26
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Bahamas, ID: 30
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Bahamas, ID: 35
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
Processing Belize
	Loaded data for Belize, ID: 0
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Belize, ID: 1
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data fo

	Loaded data for Dominican Republic, ID: 136
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 137
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 146
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 149
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 150
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 151
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 156
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 157
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for

	Loaded data for Dominican Republic, ID: 357
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 359
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 363
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 367
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 368
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 369
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 374
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 376
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for

		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 537
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 538
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 539
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 541
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 542
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 548
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 549
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Dominican Republic, ID: 550
		processing 1in5
		processing 1in50
		processing 1in

	Loaded data for Haiti, ID: 352
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 358
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 365
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 383
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 412
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 423
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 434
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 444
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 445
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded da

		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 1460
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 1464
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 1465
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 1479
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 1482
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Haiti, ID: 1485
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
Processing Jamaica
	Loaded data for Jamaica, ID: 1
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Jamaica, ID: 2
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Jamaica, ID: 3
		p

	Loaded data for Saint Kitts and Nevis, ID: 10
		processing 1in5
		processing 1in50
		processing 1in100
	Loaded data for Saint Kitts and Nevis, ID: 13
		processing 1in5
		processing 1in50
		processing 1in100
	Loaded data for Saint Kitts and Nevis, ID: 14
		processing 1in5
		processing 1in50
		processing 1in100
Processing Saint Lucia
	Loaded data for Saint Lucia, ID: 5
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 6
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 7
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 9
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 10
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
	Loaded data for Saint Lucia, ID: 13
		processing 1in5
		processing 1in50
		processing 1in100


In [138]:
results_all_pop_df = pd.concat(results_all_pop, sort=False)

In [139]:
results_all_pop_df.head()

min depth return period      2000      2005      2010  \
    country                                                                     
ATG Antigua and Barbuda        >0          1in5  0.031747  0.032304  0.032592   
    Antigua and Barbuda     >15cm          1in5  0.014207  0.014148  0.013775   
    Antigua and Barbuda       >1m          1in5  0.003844  0.003585  0.003523   
    Antigua and Barbuda     >50cm          1in5  0.007579  0.007258  0.006925   
    Antigua and Barbuda        >0         1in50  0.159355  0.161596  0.162677   

                             2015      2020  
    country                                  
ATG Antigua and Barbuda  0.033130  0.032947  
    Antigua and Barbuda  0.013645  0.013217  
    Antigua and Barbuda  0.003579  0.003421  
    Antigua and Barbuda  0.006926  0.006557  
    Antigua and Barbuda  0.162655  0.162775

In [140]:
results_all_pop_df.to_csv("urban_population_exposed_to_floods_AUG7.csv")

### C)  Agriculture Exposure
1. Loop through each return period
2. Define flooded areas with several min depth (see flood_bins)
3. Summarize agricultural area exposed

In [141]:
ag_dict = {
    0:'non-cropland',
    1:'cropland'
}
ag_dict_flood = {
    0:'non',
    1:'crop_flooded'
}

In [142]:
global_admin = '/home/public/Data/GLOBAL/ADMIN/g2015_0_simplified.shp'
adm0 = gpd.read_file(global_admin)

In [143]:
ag_path = '/home/public/Data/GLOBAL/LANDCOVER/AG/GFSAD1KCM.2010.001.2016348142550.tif'
ag_rio = rio.open(ag_path)

In [162]:
results_all_ag = {}

In [172]:
selected_countries = {
#     'ATG':'Antigua and Barbuda',
#     'BHS':'Bahamas',
#     'BLZ':'Belize',
#     'BRB':'Barbados',
#     'DMA':'Dominica',
#     'DOM':'Dominican Republic',
#     'GRD':'Grenada',
#     'GUY':'Guyana',
#     'HTI':'Haiti',
#     'JAM':'Jamaica',
    'KNA':'Saint Kitts and Nevis',
    'LCA':'Saint Lucia',
    'SUR':'Suriname',
    'MAF':'Saint-Martin', # french
    'SXM':'Sint Maarten', # dutch
#     'TCA':'Turks and Caicos Islands', #### DIDN'T RECEIVE WSF DATA
    'TTO':'Trinidad and Tobago',
    'VCT':'Saint Vincent and the Grenadines'    
}

In [173]:
for iso3, country in selected_countries.items():
    print(f"Processing {country}")
    out_dir = os.path.join(DATA_DIR,iso3)
    out_tif = os.path.join(out_dir, "combo.tif")
    cur_tif = rio.open(out_tif)

    # Load metadata and urban extents
    data_bands = raster_stack_meta[iso3]
    dst_transform = cur_tif.transform

    # to store country results
    results_ag = {}
    zs_crop_country = []
    for period in periods:
        results_ag[period] = []
    
    # iterate by polygon, much more efficient since they are islands
    adm0_sel = adm0.loc[adm0.ISO3166_1_==iso3]
    if type(adm0_sel.geometry.iloc[0]) == Polygon:
        polys = [adm0_sel.geometry.iloc[0]]
    else:
        polys = list(adm0_sel.geometry.iloc[0])
    
    for idx, poly in enumerate(polys):
        
        geometry = poly
        # get pixel coordinates of the geometry's bounding box
        ul = cur_tif.index(*geometry.bounds[0:2])
        lr = cur_tif.index(*geometry.bounds[2:4])
        window = ((float(lr[0]), float(ul[0]+1)), (float(ul[1]), float(lr[1]+1)))
        t = dst_transform
        shifted_affine = Affine(t.a, t.b, t.c+ul[1]*t.a, t.d, t.e, t.f+lr[0]*t.e)

        # reorganize data
        data = {
                'flood data':{
                    '1in5': {
                        'fluvial': cur_tif.read(data_bands[('flood data','1in5','fluvial')], window=window) if ('flood data','1in5','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in5','pluvial')], window=window) if ('flood data','1in5','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in5','coastal')], window=window) if ('flood data','1in5','coastal') in data_bands else None
                    },
                    '1in50':{
                        'fluvial': cur_tif.read(data_bands[('flood data','1in50','fluvial')], window=window) if ('flood data','1in50','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in50','pluvial')], window=window) if ('flood data','1in50','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in50','coastal')], window=window) if ('flood data','1in50','coastal') in data_bands else None
                    },
                    '1in100': {
                        'fluvial': cur_tif.read(data_bands[('flood data','1in100','fluvial')], window=window) if ('flood data','1in100','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in100','pluvial')], window=window) if ('flood data','1in100','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in100','coastal')], window=window) if ('flood data','1in100','coastal') in data_bands else None
                    },
                    '1in1000': {
                        'fluvial': cur_tif.read(data_bands[('flood data','1in1000','fluvial')], window=window) if ('flood data','1in1000','fluvial') in data_bands else None,
                        'pluvial': cur_tif.read(data_bands[('flood data','1in1000','pluvial')], window=window) if ('flood data','1in1000','pluvial') in data_bands else None,
                        'coastal': cur_tif.read(data_bands[('flood data','1in1000','coastal')], window=window) if ('flood data','1in1000','coastal') in data_bands else None
                    }
                }
            }

        print(f"\tLoaded data for {country}, polygon {idx}")
        
        # Load ag data (from global file)
        ul = ag_rio.index(*geometry.bounds[0:2])
        lr = ag_rio.index(*geometry.bounds[2:4])
        window = ((float(lr[0]), float(ul[0]+1)), (float(ul[1]), float(lr[1]+1)))
        target_shape = data['flood data']['1in5']['coastal'].shape
#         if data['flood data']['1in5']['pluvial'] is not None:
#             target_shape = data['flood data']['1in5']['pluvial'].shape
#         else:
#             target_shape = data['flood data']['1in5']['coastal'].shape
#             print("could not find target shape")
        ag_data = ag_rio.read(
            1, window=window,
            out_shape=(1, target_shape[0], target_shape[1]),
            resampling=Resampling.nearest
          )
        ag_data = ag_data.astype('float32')
        ag_data[ag_data==0] = -1
        ag_data[ag_data==9] = 0
        ag_data[ag_data>0] = 1
        ag_data = np.ma.array(ag_data, mask=(ag_data==-1), fill_value=-9999)
        
        for period in periods:

            if (pluvial is None and fluvial is None) and period =='1in1000': # if no fathom data can't run this RP
                continue

            print(f"\t\tprocessing {period}")

            # Prepare flood data
            fluvial = data['flood data'][period]['fluvial']
            pluvial = data['flood data'][period]['pluvial']
            coastal = data['flood data'][period]['coastal']

            if pluvial is None and fluvial is None:
                coastal[(coastal == -99999.0)] = -9999
                max_depth = coastal.copy()
            else:
                if pluvial is not None:
                    pluvial[(pluvial == -9999.) | (pluvial == 999)] = -9999
                if fluvial is not None:
                    fluvial[(fluvial == -9999.) | (fluvial == 999)] = -9999
                    max_fathom = np.maximum(fluvial, pluvial)
                else:
                    max_fathom = pluvial.copy()

                if coastal is not None:
                    coastal[(coastal == -99999.0)] = -9999
                    max_depth = np.maximum(max_fathom, coastal)
                else:
                    max_depth = max_fathom.copy()
            max_depth_ma = np.ma.array(max_depth, mask=(max_depth==-9999), fill_value=-9999)
            
            if (max_depth_ma.shape != ag_data.shape):
                print("error: datasets do not have the same shape")
            
            results_dict = {}
            # for each min flood, get sum of cells intersecting flood categories
            for key, label in flood_bins.items():
                flood_bool = (max_depth_ma>key)*1
                flood_ag_intersect = ag_data*flood_bool
                zs = pd.DataFrame(zonal_stats(poly, flood_ag_intersect.filled(), affine=shifted_affine, category_map=ag_dict_flood, categorical=True, nodata=-9999))
                zs.index = [idx]
                results_dict[label] = zs

            results_df = pd.concat(results_dict, sort=False)
            results_df.fillna(0, inplace = True)

            results_ag[period].append(results_df)
            
        # prepare sum of cropland up to get shares
        zs_tot = pd.DataFrame(zonal_stats(poly, ag_data.filled(), affine=shifted_affine, categorical=True, category_map=ag_dict, nodata=-9999))
        zs_tot.fillna(0, inplace = True)
        zs_tot.index = [idx]
        zs_crop_country.append(zs_tot)

    # AGGREGATE TO COUNTRY-LEVEL

    # convert list of df into one df
    for key, item in results_ag.items():
        if (pluvial is None and fluvial is None) and key =='1in1000':
            results_ag[key] = None
            continue
        results_ag[key] = pd.concat(results_ag[key]).groupby(level=0).sum()
    results_ag_df = pd.concat(results_ag, sort=False)
    
    for ag_cat in ag_dict_flood.values():
        if ag_cat not in results_ag_df.columns:
            results_ag_df[ag_cat] = 0
    results_ag_df = results_ag_df[ag_dict_flood.values()]
    results_ag_df.index.names = ['return period', 'min depth']

    results_ag_df['country'] = country
    results_ag_df = results_ag_df.reset_index()
    results_ag_df = results_ag_df.set_index(['country', 'return period', 'min depth'])

    # aggregate sum of cropland
    results_tot = pd.DataFrame(pd.concat(zs_crop_country).sum(), columns=[country]).transpose()
    for ag_cat in ag_dict.values():
        if ag_cat not in results_tot.columns:
            results_tot[ag_cat] = 0
    results_tot = results_tot[['cropland']]
    results_tot.index.name = 'country'

    #get shares
    results_pct = results_ag_df.join(results_tot)
    results_pct = results_pct.reset_index(drop=False).set_index('country')
    results_pct['crop_flooded_pct'] = results_pct['crop_flooded']/results_pct['cropland']
    results_all_ag[iso3] = results_pct

Processing Saint Kitts and Nevis
	Loaded data for Saint Kitts and Nevis, polygon 0
could not find target shape
		processing 1in5
		processing 1in50
		processing 1in100
	Loaded data for Saint Kitts and Nevis, polygon 1
could not find target shape
		processing 1in5
		processing 1in50
		processing 1in100
Processing Saint Lucia
	Loaded data for Saint Lucia, polygon 0
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
Processing Suriname
	Loaded data for Suriname, polygon 0
		processing 1in5
		processing 1in50
		processing 1in100
		processing 1in1000
Processing Saint-Martin
	Loaded data for Saint-Martin, polygon 0
could not find target shape
		processing 1in5
		processing 1in50
		processing 1in100
	Loaded data for Saint-Martin, polygon 1
could not find target shape
		processing 1in5
		processing 1in50
		processing 1in100
Processing Sint Maarten
	Loaded data for Sint Maarten, polygon 0
could not find target shape
		processing 1in5
		processing 1in50
		processing 1i

In [174]:
results_all_ag_df = pd.concat(results_all_ag, sort=False)

In [175]:
results_all_ag_df.to_csv("cropland_exposed_to_floods_AUG7.csv")